In [1]:
import openai
from dotenv import load_dotenv
import os

# Load OpenAI API key from .env file
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [11]:
from datasets import load_dataset
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack import Pipeline
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder


# Create a RAG query pipeline
prompt_template = """
    Given these documents, answer the question.\nDocuments:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \nQuestion: {{question}}
    \nAnswer:
"""

rag_pipeline = Pipeline()
rag_pipeline.add_component(instance=InMemoryBM25Retriever(document_store=InMemoryDocumentStore()), name="retriever")
rag_pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
rag_pipeline.add_component(instance=OpenAIGenerator(), name="llm")
rag_pipeline.add_component(instance=AnswerBuilder(), name="answer_builder")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")
rag_pipeline.connect("llm.replies", "answer_builder.replies")
rag_pipeline.connect("retriever", "answer_builder.documents")

# Draw the pipeline
rag_pipeline.draw("./rag_pipeline.png")

151

In [ ]:
documents = [Document(content="There are over 7,000 languages spoken around the world today."),
			       Document(content="Elephants have been observed to behave in a way that indicates a high level of self-awareness, such as recognizing themselves in mirrors."),
			       Document(content="In certain parts of the world, like the Maldives, Puerto Rico, and San Diego, you can witness the phenomenon of bioluminescent waves.")]
rag_pipeline.get_component("retriever").document_store.write_documents(documents)

In [12]:
question = "How many languages are there?"
result = rag_pipeline.run(
            {
                "retriever": {"query": question},
                "prompt_builder": {"question": question},
                "answer_builder": {"query": question},
            }
        )
print(result['answer_builder']['answers'][0].query)
print(result['answer_builder']['answers'][0].data)
print(result['answer_builder']['answers'][0].documents)

How many languages are there?
There are over 7,000 languages spoken around the world today.
[Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.', score: 14.697908314805233), Document(id=809bb7b9cdd0ffc53a138ae56f78b765a8b2fc70d771728702a950476e2464a5, content: '[28] It is possible to go diving and see the ruins. The secretariat of the UNESCO Convention on the ...', meta: {'url': 'https://en.wikipedia.org/wiki/Lighthouse_of_Alexandria', '_split_id': 9}, score: 11.402664863437517), Document(id=bb2c5f3d2e2e2bf28d599c7b686ab47ba10fbc13c07279e612d8632af81e5d71, content: 'The Roman writer Pliny the Elder, writing in the first century AD, argued that the Great Pyramid had...', meta: {'url': 'https://en.wikipedia.org/wiki/Great_Pyramid_of_Giza', '_split_id': 16}, score: 9.948798320669107), Document(id=0b3ec68a3754c399d2160c93c36ea172c318edad9310828c7d056392956f884f, content: 'The building was rec

In [ ]:
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]
rag_pipeline.get_component("retriever").document_store.write_documents(docs)

In [14]:
question = "How old is great pyramid of Giza?"
result = rag_pipeline.run(
            {
                "retriever": {"query": question},
                "prompt_builder": {"question": question},
                "answer_builder": {"query": question},
            }
        )
print(result['answer_builder']['answers'][0].query)
print(result['answer_builder']['answers'][0].data)
print(result['answer_builder']['answers'][0].documents)

How old is great pyramid of Giza?
The Great Pyramid of Giza is approximately 4600 years old.
[Document(id=76faac9079190122b6af0e7a46f8ad79d04f8243aee0fe9fea8c999bc2dfea19, content: 'The Great Pyramid of Giza[a] is the largest Egyptian pyramid and the tomb of Fourth Dynasty pharaoh ...', meta: {'url': 'https://en.wikipedia.org/wiki/Great_Pyramid_of_Giza', '_split_id': 0}, score: 12.873353552923584), Document(id=e42388e23323dad64d9cd5dd5e93fa1391d93be3dc161236d789e2eb7de99546, content: 'These were entrusted to a boat builder, Haj Ahmed Yusuf, who worked out how the pieces fit together....', meta: {'url': 'https://en.wikipedia.org/wiki/Great_Pyramid_of_Giza', '_split_id': 60}, score: 12.228349913056208), Document(id=62b740443778bd783e082734cf3b57684f1ff204afcb42f473f32987ee79f9e0, content: '[79] The bedrock reaches a height of almost 6 metres (20 ft) above the pyramid base at the location ...', meta: {'url': 'https://en.wikipedia.org/wiki/Great_Pyramid_of_Giza', '_split_id': 22}, score: 1